In [2]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import LabelEncoder



In [4]:
#Settings
%matplotlib inline
pd.set_option('display.max_columns', 97)

In [6]:
def readFeatures():
    
    rawData = pd.read_csv("2016-SFO-Customer-Survey-Data.csv")
    
    data = rawData.drop(["*RESPNUM","CCGID", "RUNID", "DAY", "INTDATE", "EGYPTAIR", "SAQ", "DEST CODE", 
                         "AIRLINE CODE", "STRATA", "PEAK", "GATE", "METHOD", "FLIGHT", "LANG", "WEIGHT", 
                         "FLIGHT", "DESTGEO","Q3GETTO1", "Q3GETTO2", "Q3GETTO3", "Q3PARK", "Q8COM", "Q8COM2", 
                         "Q8COM3", "Q8COM4", "Q8COM5", "Q9COM", "Q9COM2", "Q9COM3", "Q10COM1", "Q10COM2", 
                         "Q10COM3", "Q12PRECOM1", "Q12PRECOM2", "Q12PRECOM3", "Q15COM1", "Q15COM2", "Q15COM3", 
                         "ARRTIME", "DEPTIME"], axis = 1)
    
    return data